In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

### Modify the parameters before running
---


In [2]:
original_data_path = './../sensor_scaled_cleaned.csv'
cherrypick_test_path = './data/features_broken_scaled.csv'
window_size = 37
flattened_size = 1369

---

In [3]:
data_final = pd.read_csv(original_data_path, index_col='index')

In [4]:
broken_idx = data_final[data_final['machine_status'] == 'BROKEN'].index

In [5]:
# supplemental function to check if indices in a window are consecutive
def checkConsecutive(seq):
    n = len(seq) - 1
    return (sum(np.diff(seq) == 1) >= n)

In [6]:
anom_sample = np.empty((0, flattened_size))
anom_sample_index = []

In [7]:
def construct_features_anom(num, broken):
    global anom_sample
    global broken_idx
    
    temp_anom = data_final.loc[lambda df: df.index >= broken - window_size + 1, :]
    
    for idx in range(0, window_size):
        temp = temp_anom.iloc[idx: idx + window_size]
        if checkConsecutive(temp.index.to_numpy()):
            if temp['machine_status'].str.contains('BROKEN').any():
                anom_sample = np.append(anom_sample, temp.drop(['timestamp', 'machine_status'], axis=1).to_numpy().reshape(1, flattened_size), axis=0)  
                anom_sample_index.append(num)
        

In [8]:
for idx, i in enumerate(broken_idx):
    construct_features_anom(idx, i)

In [9]:
with open(cherrypick_test_path, 'ab') as f:
    np.savetxt(f, anom_sample, delimiter=',', newline='\n')